In [1]:
import os
import time
import torch
import sys
import torch.optim as optim
from torchvision import transforms as T
%matplotlib notebook

In [2]:
from models import SpecialFuseNetModel
from data_manager import rgbd_gradients_dataset, rgbd_gradients_dataloader
from train import FuseNetTrainer

In [3]:
CWD             = os.getcwd()
DATASET_DIR     = os.path.join(CWD,'data/nyuv2')
print(DATASET_DIR)

C:\Users\tomav\Documents\GitHub\cs236781-project\data/nyuv2


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [5]:
IMAGE_SIZE       = (224,224)
TRAIN_TEST_RATIO = 0.5#0.9
BATCH_SIZE       = 4
NUM_WORKERS      = 4

BETAS        = (0.9,0.99)
LR           = 0.001
MOMENTUM     = 0.9
WEIGHT_DECAY = 0.0005

In [6]:
rgb_tf = T.Compose([
    # Resize to constant spatial dimensions
    T.Resize(IMAGE_SIZE),
    # PIL.Image -> torch.Tensor
    T.ToTensor(),
    # Dynamic range [0,1] -> [-1, 1]
    T.Normalize(mean=(.5,.5,.5), std=(.5,.5,.5)),
])
depth_tf = T.Compose([
    # Resize to constant spatial dimensions
    T.Resize(IMAGE_SIZE),
    # PIL.Image -> torch.Tensor
    T.ToTensor(),
    # Dynamic range [0,1] -> [-1, 1]
    T.Normalize(mean=(.5,), std=(.5,)),
])

In [7]:
rgbd_grads_ds = rgbd_gradients_dataset(root=DATASET_DIR,rgb_transforms=rgb_tf,depth_transforms=depth_tf)

In [8]:
dl_train,dl_test = rgbd_gradients_dataloader(root=DATASET_DIR,
                                             batch_size=BATCH_SIZE,
                                             num_workers=NUM_WORKERS,
                                             train_test_ration=TRAIN_TEST_RATIO,
                                             rgb_transforms=rgb_tf,depth_transforms=depth_tf)

In [9]:
# _ = plot.rgbd_gradients_dataset_first_n(dataset=rgbd_grads_ds,n=5)
print(f'Found {len(rgbd_grads_ds)} images in dataset folder.')

Found 2 images in dataset folder.


In [10]:
sample_batch = next(iter(dl_train))
rgb_size = tuple(sample_batch['rgb'].shape[1:])
depth_size = tuple(sample_batch['depth'].shape[1:])
grads_size = tuple(sample_batch['x'].shape[1:])

In [11]:
fusenetmodel = SpecialFuseNetModel(rgb_size=rgb_size,depth_size=depth_size,grads_size=grads_size, device=device)

[I] - default optimizer set: SGD(lr=0.001,momentum=0.9,weight_decay=0.0005)
[I] - default scheduler set: StepSR(step_size=1000,gamma=0.1)


In [12]:
trainer = FuseNetTrainer(model=fusenetmodel, device=device)

In [13]:
checkpoint_file = 'checkpoints/special_fusenet'
if os.path.isfile(f'{checkpoint_file}.pt'):
    os.remove(f'{checkpoint_file}.pt')

In [14]:
res = trainer.fit(dl_train, dl_test, early_stopping=400, print_every=10,
                  checkpoints=checkpoint_file)

--- EPOCH 1/400 ---
test_batch (Avg. Loss 0.014: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 2
test_batch (Avg. Loss 0.014: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.04s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 3
test_batch (Avg. Loss 0.014: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 4
test_batch (Avg. Loss 0.014: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.03s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 5
test_batch (Avg. Loss 0.014: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.04s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 6
test_batch (Avg. Loss 0.014: 100%|██████

test_batch (Avg. Loss 0.012: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.02s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 27
test_batch (Avg. Loss 0.012: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 28
test_batch (Avg. Loss 0.012: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 29
test_batch (Avg. Loss 0.012: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 30
--- EPOCH 31/400 ---
test_batch (Avg. Loss 0.011: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 31
test_batch (Avg. Loss 0.011: 100%|

test_batch (Avg. Loss 0.010: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 53
test_batch (Avg. Loss 0.010: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 54
test_batch (Avg. Loss 0.010: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.12s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 55
test_batch (Avg. Loss 0.010: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.25s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 56
test_batch (Avg. Loss 0.013: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.09s/it]
--- EPOCH 61/400 ---
test_batch (Avg. Loss 1.076: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<0

test_batch (Avg. Loss 1.077: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
--- EPOCH 91/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.08s/it]
--- EPOCH 101/400 ---
test_batch (Avg. Loss 1.077: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.04s/it]
--- EPOCH 111/400 ---
train_batch (Avg. Loss 0.008: 100%|██████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.16s/it]


test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
--- EPOCH 121/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
--- EPOCH 131/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
--- EPOCH 141/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.02s/it]
--- EPOCH 151/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.13s/it]


...

test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
--- EPOCH 261/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.03s/it]
--- EPOCH 271/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]
--- EPOCH 281/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.06s/it]


test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.06s/it]
--- EPOCH 291/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.03s/it]
--- EPOCH 301/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.06s/it]
--- EPOCH 311/400 ---
train_batch (Avg. Loss 0.007: 100%|██████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.16s/it]


test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.07s/it]
--- EPOCH 321/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.04s/it]
--- EPOCH 331/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.03s/it]
--- EPOCH 341/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.07s/it]
--- EPOCH 351/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.04s/it]


test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.08s/it]
--- EPOCH 361/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.08s/it]
--- EPOCH 371/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.06s/it]
--- EPOCH 381/400 ---
train_batch (Avg. Loss 0.007: 100%|██████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.20s/it]


test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.11s/it]
--- EPOCH 391/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.11s/it]
--- EPOCH 400/400 ---
test_batch (Avg. Loss 1.078: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.07s/it]
